In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [3]:
device='mps'

In [4]:
X1=torch.rand(10000,1)
X2=torch.rand(10000,1)
X3=torch.rand(10000,1)
X1.shape, X2.shape, X3.shape

(torch.Size([10000, 1]), torch.Size([10000, 1]), torch.Size([10000, 1]))

In [5]:
Y1=((X1+X2+X3)<1).float()
Y2=((1<(X1+X2+X3))&(X1+X2+X3<2)).float()
Y3=((X1+X2+X3)>2).float()
Y1.shape, Y2.shape, Y3.shape

(torch.Size([10000, 1]), torch.Size([10000, 1]), torch.Size([10000, 1]))

In [6]:
Data=torch.cat([X1,X2,X3,Y1,Y2,Y3],dim=1)
Data=Data.to(device)
Data.shape

torch.Size([10000, 6])

In [7]:
Data

tensor([[0.4101, 0.0504, 0.0263, 1.0000, 0.0000, 0.0000],
        [0.7876, 0.5624, 0.6420, 0.0000, 1.0000, 0.0000],
        [0.0931, 0.0392, 0.3370, 1.0000, 0.0000, 0.0000],
        ...,
        [0.5517, 0.1842, 0.2109, 1.0000, 0.0000, 0.0000],
        [0.6635, 0.9509, 0.0897, 0.0000, 1.0000, 0.0000],
        [0.6127, 0.7719, 0.4613, 0.0000, 1.0000, 0.0000]], device='mps:0')

In [8]:
train_size=int(len(Data)*0.7)
test_size=len(Data)-train_size
Data=Data[torch.randperm(Data.size(0)),:]
train_Data=Data[:train_size,:]
test_Data=Data[train_size:,:]
train_Data.shape, test_Data.shape

(torch.Size([7000, 6]), torch.Size([3000, 6]))

In [9]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN,self).__init__()
        set.net=nn.Sequential(
            nn.Linear(3,5), nn.ReLU(),
            nn.Linear(5,5), nn.ReLU(),
            nn.Linear(5,5), nn.ReLU(),
            nn.Linear(5,3)
        )
    def forward(self,x):
        y=self.net(x)
        return y

In [10]:
model=DNN().to(device)
model

TypeError: cannot set 'net' attribute of immutable type 'set'